# Managing experiments with MLflow client

In this notebook, we will explore how to manage experiments using the MLflow client. MLflow client API provides fine-grained control over experiments, runs, and models compared to the high-level MLflow functions. MLflow client provides a Python API that allows us to programmatically manage experiments, including creating, listing, and deleting experiments. This is particularly useful for automating ML workflows and integrating MLflow with other tools.

In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

### Setting up the MLflow client
Before we begin managing experiments, we need to set up the MLflow client. We will create an instance of MlflowClient. The `mlflow.client.MlflowClient` class is the central point for managing experiments and runs in MLflow. With `MlflowClient`, we can interact with the MLflow tracking server to perform various operations on experiments.

In [2]:
# Create an MLflow client instance
client = MlflowClient()

### Creating an experiment
To create a new experiment, we can use the `create_experiment()` method of the `MlflowClient` class. This method requires the name of the experiment and optionally takes a location where artifacts are stored.

In [3]:
# Create a new experiment
experiment_id = client.create_experiment(name="My_New_Experiment")

print(f"Created experiment with ID: {experiment_id}")

Created experiment with ID: 959268168270051133


- **`create_experiment(name)`**: Creates a new experiment with the specified name and returns experiment id. If the experiment already exists, it will return an error unless we handle it.
> **Note**: If an experiment with the same name already exists, MLflow will raise an error. To check if an experiment exists, we can list experiments or search by name.

#### Optional: Checking if an experiment exists
Before creating a new experiment, it's good practice to check if an experiment with the same name already exists.

In [4]:
# Check if experiment exists, if not, create it
experiment = client.get_experiment_by_name("My_New_Experiment")
if experiment is None:
    experiment_id = client.create_experiment("My_New_Experiment")
else:
    experiment_id = experiment.experiment_id

print(f"Using experiment with ID: {experiment_id}")

Using experiment with ID: 959268168270051133


- **`get_experiment_by_name(name)`**: Retrieve an existing experiment based on its specified name. It takes the experiment name as an argument and returns an experiment object if found. If the experiment does not exist, it will return a None value.

## Listing experiments and experiment details
MLflow allows us to list all experiments in the tracking server. This can be helpful for finding the ID of an experiment or checking which experiments are available.

In [5]:
# List all experiments
experiments_after_creating = client.search_experiments()

# Display experiment names and IDs
for experiment in experiments_after_creating:
    print(f"Experiment ID: {experiment.experiment_id}, Name: {experiment.name}, Artifact location: {experiment.artifact_location}")

Experiment ID: 959268168270051133, Name: My_New_Experiment, Artifact location: file:///C:/Users/israe/Documents/Codes/Notebooks/mlruns/959268168270051133
Experiment ID: 678057457486770417, Name: MLflow Model Registry Example, Artifact location: file:///C:/Users/israe/Documents/Codes/Notebooks/mlruns/678057457486770417
Experiment ID: 824640624065164964, Name: Iris Classification Experiment2, Artifact location: file:///C:/Users/israe/Documents/Codes/Notebooks/mlruns/824640624065164964
Experiment ID: 191308692135956385, Name: Iris Classification Experiment, Artifact location: file:///C:/Users/israe/Documents/Codes/Notebooks/mlruns/191308692135956385
Experiment ID: 328990853911858228, Name: California Housing Regression Autologging, Artifact location: file:///C:/Users/israe/Documents/Codes/Notebooks/mlruns/328990853911858228


- **`search_experiments()`**: Returns a list of all active experiments in the tracking server, including their IDs, names, and artifact locations.

### Deleting an experiment
If we need to delete an experiment, we can use the `delete_experiment()` method. This marks the experiment as deleted but does not remove its data from the tracking server.

In [6]:
# Delete an experiment by ID
client.delete_experiment(experiments_after_creating[0].experiment_id)
print(f"Experiment with ID: {experiments_after_creating[0].experiment_id} has been deleted.")

# Display experiment names and IDs
experiments_after_deletion = client.search_experiments()
print("\nExperiments list after deletion:")
for experiment in experiments_after_deletion:
    print(f"Experiment ID: {experiment.experiment_id}, Name: {experiment.name}")

Experiment with ID: 959268168270051133 has been deleted.

Experiments list after deletion:
Experiment ID: 678057457486770417, Name: MLflow Model Registry Example
Experiment ID: 824640624065164964, Name: Iris Classification Experiment2
Experiment ID: 191308692135956385, Name: Iris Classification Experiment
Experiment ID: 328990853911858228, Name: California Housing Regression Autologging


- **`delete_experiment()`**: Marks an experiment and its associated metadata, runs, metrics, parameters, and tags as deleted. Deleted experiments are not permanently removed; they are simply hidden from the list of active experiments. This function takes a single parameter, which is the `experiment_id`. This ID is a string or integer that uniquely identifies the experiment we want to delete.

> **Note**: Deleting an experiment does not delete its associated runs or artifacts. They remain accessible if we know the run IDs.

### Restoring a deleted experiment
MLflow allows us to restore a deleted experiment using the `restore_experiment()` method, which moves it back to an active state.

In [7]:
# Restore a deleted experiment by ID
client.restore_experiment(experiments_after_creating[0].experiment_id)
print(f"Experiment with ID: {experiments_after_creating[0].experiment_id} has been restored.")

# Display experiment names and IDs
experiments_after_restoring = client.search_experiments()
print("\nExperiments list after restoring:")
for experiment in experiments_after_restoring:
    print(f"Experiment ID: {experiment.experiment_id}, Name: {experiment.name}")

Experiment with ID: 959268168270051133 has been restored.

Experiments list after restoring:
Experiment ID: 959268168270051133, Name: My_New_Experiment
Experiment ID: 678057457486770417, Name: MLflow Model Registry Example
Experiment ID: 824640624065164964, Name: Iris Classification Experiment2
Experiment ID: 191308692135956385, Name: Iris Classification Experiment
Experiment ID: 328990853911858228, Name: California Housing Regression Autologging


- **`restore_experiment()`**: Restores a deleted experiment, making it active again. This function also takes a single parameter, `experiment_id`, which specifies the ID of the experiment we want to restore.

### Renaming an experiment
MLflow also allows us to rename an experiment if we need to update its name.

In [8]:
# Rename an experiment by ID
client.rename_experiment(experiments_after_creating[0].experiment_id, "Renamed_Experiment")
print(f"Experiment ID: {experiments_after_creating[0].experiment_id} has been renamed to: Renamed_Experiment")

Experiment ID: 959268168270051133 has been renamed to: Renamed_Experiment


- **`rename_experiment()`**: Renames an experiment by its ID. This function requires two parameters: the `experiment_id`, which is the ID of the experiment we want to rename, and `new_name`, which is a string representing the new name we want to assign to the experiment. It doesn't return any value but successfully changes the experiment’s name in the MLflow system.

## Searching experiments
The `mlflow.search_experiments()` function allows us to search for experiments in the MLflow tracking server based on specific criteria. This function is flexible and can be customized with various parameters to filter and sort experiments according to our needs. The function returns a `PagedList` of `Experiment` objects that match the query and we can access their properties like `experiment_id`, `name`, `tags`, and more.


#### Search active experiments
In this example, we search for all active experiments and print their IDs and names.

In [9]:
# Search for all active experiments
experiments = mlflow.search_experiments(view_type=ViewType.ACTIVE_ONLY)

# Display experiment names
for exp in experiments:
    print(f"Experiment ID: {exp.experiment_id}, Name: {exp.name}")

Experiment ID: 959268168270051133, Name: Renamed_Experiment
Experiment ID: 678057457486770417, Name: MLflow Model Registry Example
Experiment ID: 824640624065164964, Name: Iris Classification Experiment2
Experiment ID: 191308692135956385, Name: Iris Classification Experiment
Experiment ID: 328990853911858228, Name: California Housing Regression Autologging


- **`view_type`**: This parameter specifies which experiments to include in the search results. we can use:
  - `ViewType.ACTIVE_ONLY`: Only active experiments.
  - `ViewType.DELETED_ONLY`: Only deleted experiments.
  - `ViewType.ALL`: All experiments, both active and deleted.

#### Filter experiments by name
Here, we filter experiments by their name. The query returns experiments where the name is exactly `'Iris Classification Experiment'`.

In [10]:
# Search for an experiment with a specific name
experiments = mlflow.search_experiments(
    view_type=ViewType.ALL,
    filter_string="name = 'Iris Classification Experiment'"
)

# Display matching experiments
for exp in experiments:
    print(f"Experiment ID: {exp.experiment_id}, Name: {exp.name}")

Experiment ID: 191308692135956385, Name: Iris Classification Experiment


- **`filter_string`**: A query string to filter the experiments based on specific criteria, such as name, creation time, last update time, or tags.

#### Order experiments by creation time
In this example, we search for all experiments and sort them by their creation time in ascending order.

In [11]:
# Search for experiments and order them by creation time in ascending order
experiments = mlflow.search_experiments(
    view_type=ViewType.ALL,
    order_by=["creation_time ASC"],
    max_results = 3    
)

# Display experiment names and creation times
for exp in experiments:
    print(f"Experiment ID: {exp.experiment_id}, Name: {exp.name}, Created At: {exp.creation_time}")

Experiment ID: 328990853911858228, Name: California Housing Regression Autologging, Created At: 1724614767818
Experiment ID: 191308692135956385, Name: Iris Classification Experiment, Created At: 1724749168776
Experiment ID: 824640624065164964, Name: Iris Classification Experiment2, Created At: 1724856983795


- **`order_by`**: A list of columns to order the results by. We can specify whether the sorting should be ascending (`ASC`) or descending (`DESC`). The default is to sort by `last_update_time DESC`.
- **`max_results`**: The maximum number of experiments to return. If not specified, it defaults to 1000. Be aware that some server backends might enforce their own limits. If our search returns more experiments than `max_results`, the results will be paginated. We can retrieve subsequent pages using the `page_token` from the first set of results.
- **`page_token`**: If our query results are paginated, this token allows you to retrieve the next page of results.

#### Filter by experiment tags
This example demonstrates filtering experiments by a tag. We search for experiments tagged with `team = 'data-science'`.

In [12]:
# Search for experiments with a specific tag
experiments = mlflow.search_experiments(
    view_type=ViewType.ALL,
    filter_string="tags.`team` = 'data-science'"
)

# Display matching experiments
if not experiments:
  print("No experiments found.")
for exp in experiments:
    print(f"Experiment ID: {exp.experiment_id}, Name: {exp.name}, Tags: {exp.tags}")

No experiments found.
